<a href="https://colab.research.google.com/github/HiTECHFullOn/ET2iCal/blob/main/everytime_to_iCal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!pip install vobject

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.3/448.3 kB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for vobject: filename=vobject-0.9.6.1-py3-none-any.whl size=47680 sha256=ab16ade98c609fa2d41b8371982a3d527378df8d8fc253af449353ee52a28bb3
  Stored in directory: /root/.cache/pip/wheels/50/30/da/1b11581351fd0d2c273193d7f615532f95a48dcb717af207de
Successfully built vobject


## STEP 1. 에브리타임 시간표 불러오기

In [ ]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36')

driver = webdriver.Chrome(options=options)

In [ ]:
def return_time(subject): #div.subject를 Tag로 입력받아 수업 시수와 시작시간을 시간단위로 리턴
  a = str(subject)
  height = int(a[a.find("height: ")+8 : a.find("px; ")])-2
  top = int(a[a.find("top: ")+5 : a.find("px;\">")])
  px = 50
  mins = int((height/px)*60+1)
  start = (int(((top/px)*60)//60), int(int((top/px)*60)%60))
  return [start, mins]

In [ ]:
def get_table(url):
  driver = webdriver.Chrome(options=options)
  driver.get(url)
  time.sleep(5)
  driver.save_screenshot('screenshot.png')
  html = driver.page_source
  soup= BeautifulSoup(html, 'html.parser')
  td = soup.select('#container > div > div.tablebody > table > tbody > tr > td')
  subject = soup.select('#container > div > div.tablebody > table > tbody > tr > td > div > div.subject')
  username = soup.select_one('#container > aside > div.title > h1').get_text()

  subject_cnt = [] #요일별 과목 개수

  for i in td:
    cnt = str(i).count('<h3>')
    subject_cnt.append(cnt)

  driver.quit()

  dow = ['MO','TU','WE','TH','FR','SA','SU']
  dow_list = []
  for i in range(len(subject_cnt)):
      for j in range(subject_cnt[i]):
          dow_list.append(dow[i])
  sub_list = []

  k=0
  for i in subject :
      sub=[]
      sub.append(i.find('h3').get_text())     #과목명
      sub.append(i.find('em').get_text())     #교수명
      sub.append(i.find('span').get_text())   #장소
      sub.append(return_time(i))              #시간정보
      sub.append([dow_list[k]])
      k+=1
      sub_list.append(sub)
  return [sub_list, username]

## STEP 2. 불러온 시간표를 iCal 형식으로 변환

In [ ]:
import vobject
from datetime import datetime, timedelta
semester_start = datetime(2023, 9, 1, 0, 0, 0)
semester_end = '20231215T000000'

In [ ]:
def cal(lesson, username):
  (name, desc, loca, (starttime, mins), dow) = tuple(lesson)
  global cal1
  vevent = cal1.add('vevent')
  vevent.add('summary').value = name
  vevent.add('description').value = desc

  dow_list = ['MO','TU','WE','TH','FR','SA','SU']
  k=semester_start.weekday()
  dow_list = dow_list[k:]+dow_list[:k]

  n = dow_list.index(dow[0])
  dtstart = semester_start + timedelta(days=n, hours=starttime[0], minutes=starttime[1])
  dtend = dtstart + timedelta(minutes=mins)

  vevent.add('dtstart').value = dtstart
  vevent.add('dtend').value = dtend
  vevent.add('dtstamp').value = semester_start
  vevent.add('location').value = loca
  vevent.add('rrule').value = 'FREQ=WEEKLY;INTERVAL=1;UNTIL='+semester_end+';BYDAY='+','.join(dow)

  with open(username+'.ics','wb') as f:
    f.write(cal1.serialize().encode('utf-8'))
    print('iCal 파일 덮어쓰기 완료.')

# ⏰ 🔗 ➡ 📅 😏

In [ ]:
cal1 = vobject.iCalendar()
url = input('에브리타임 시간표 링크를 입력하세요: ')
data = get_table(url)
sub_list = data[0]
username = data[1]
ID = url[21:]
for lesson in sub_list:
    cal(lesson, username)
if sub_list == []:
    print("공개되지 않은 시간표거나 잘못된 링크입니다.")
else: print('\n왼쪽의 📁폴더 아이콘을 눌러 .ics 파일을 다운로드할 수 있습니다.')

에브리타임 시간표 링크를 입력하세요: https://everytime.kr/@VkZUnl4bjmWzG0nFVlE1
iCal 파일 덮어쓰기 완료.
iCal 파일 덮어쓰기 완료.
iCal 파일 덮어쓰기 완료.
iCal 파일 덮어쓰기 완료.
iCal 파일 덮어쓰기 완료.
iCal 파일 덮어쓰기 완료.
iCal 파일 덮어쓰기 완료.
iCal 파일 덮어쓰기 완료.
iCal 파일 덮어쓰기 완료.
iCal 파일 덮어쓰기 완료.
iCal 파일 덮어쓰기 완료.

왼쪽의 📁폴더 아이콘을 눌러 .ics 파일을 다운로드할 수 있습니다.
